# This notebook is used to delete the relevant IAM roles, policy, RedShift cluster, Glue database and secret in Secret Manager.
Note: This notebook does not delete VPC, SageMaker Studio, S3, EFS.

## Variables

In [1]:
secret_name='bankdemo_redshift_login' 


## Delete RedShift cluster, Glue database, secret


In [2]:
from botocore.exceptions import ClientError
import json
import boto3
import sagemaker

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

redshift = boto3.client('redshift')
secretsmanager = boto3.client('secretsmanager')
glue = boto3.client('glue')
iam = boto3.client('iam')
sts = boto3.client('sts')
accountID = sts.get_caller_identity()["Account"]  

Load information from secret

In [3]:
session = boto3.session.Session()
region = session.region_name

try:
    get_secret_value_response = secretsmanager.get_secret_value(
            SecretId=secret_name
        )
    secret_arn=get_secret_value_response['ARN']

except ClientError as e:
    print("Error retrieving secret. Error: " + e.response['Error']['Message'])
    
else:
    # Depending on whether the secret is a string or binary, one of these fields will be populated.
    if 'SecretString' in get_secret_value_response:
        secret = get_secret_value_response['SecretString']
    else:
        secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            
secret_json = json.loads(secret)
redshift_cluster_identifier = secret_json['dbClusterIdentifier']

database_name_glue = secret_json['database_name_glue']


Delete RedShift cluster

In [4]:
try:
    response = redshift.delete_cluster(
        ClusterIdentifier=redshift_cluster_identifier,
        SkipFinalClusterSnapshot=True
    )
    
except ClientError as e:
    if e.response['Error']['Code'] == 'ClusterNotFound':
        print("Cluster does not exists. This is ok.")
    else:
        print("Unexpected error: %s" % e)

Delete Glue database

In [5]:
try:
    response = glue.delete_database(
        Name=database_name_glue
    )
    
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityNotFoundException':
        print("Glue database does not exists. This is ok.")
    else:
        print("Unexpected error: %s" % e)

Delete secret

In [6]:
try:
    response = secretsmanager.delete_secret(
        SecretId=secret_name,
        ForceDeleteWithoutRecovery=True
    )
    
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceNotFoundException':
        print("Secret does not exists. This is ok.")
    else:
        print("Unexpected error: %s" % e)

## Delete IAM roles and policies

In [7]:
role='BankDemo'

# Delete self created policy
policyname = ['BankDemo_RedshiftPolicyToS3', 'BankDemo_RedshiftPolicyToAthena']
for i in range(len(policyname)):
    policy = f'arn:aws:iam::{accountID}:policy/{policyname[i]}'
    print(policy)
    try:
        response = iam.detach_role_policy(
            RoleName=role,
            PolicyArn=policy
        )
    
    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchEntity':
            print("IAM policy not attached to the role. This is ok.")
        else:
            print("Unexpected error: %s" % e)
            
    try:
        response = iam.delete_policy(
            PolicyArn=policy
        )

    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchEntityException':
            print("IAM policy does not exists. This is ok.")
        else:
            print("Unexpected error: %s" % e)

# Delete AWS policy
policyname = ['SecretsManagerReadWrite', 'AmazonRedshiftFullAccess', 'AmazonSageMakerFullAccess']
for i in range(len(policyname)):
    policy = f'arn:aws:iam::aws:policy/{policyname[i]}'
    print(policy)
    try:
        response = iam.detach_role_policy(
            RoleName=role,
            PolicyArn=policy
        )
    
    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchEntity':
            print("IAM policy not attached to the role. This is ok.")
        else:
            print("Unexpected error: %s" % e)
        

arn:aws:iam::138604873012:policy/BankDemo_RedshiftPolicyToS3
arn:aws:iam::138604873012:policy/BankDemo_RedshiftPolicyToAthena
arn:aws:iam::aws:policy/SecretsManagerReadWrite
arn:aws:iam::aws:policy/AmazonRedshiftFullAccess
arn:aws:iam::aws:policy/AmazonSageMakerFullAccess
